In [1]:
import numpy as np
from scipy.integrate import odeint
import paho.mqtt.client as mqtt
import time
import socket

In [2]:
def create_file_model():
    f= open("Model.py","w+")
    
    user_model = ("        Q = 2.0  \r" +
                  "        d = 1.5  \r" +
                  "        Omega = 0.65 \r" +
                  "        theta, omega = y  \r" + 
                  "        derivs = [omega,\r" +
                  "                 -omega/Q + np.sin(theta) + d*np.cos(Omega*t)]\r" +
                  "        return derivs")
    
    model = ("import numpy as np \r\n" +
             "class model:\r"
             "    def __init__(self):\r" +
             "        print('Init model')\r\n" +
             "    def function(y, t): \r") + user_model
    
    f.write(model)
    return  [1, 2]


def solver():
    # Initial values
    theta0 = 0.0     # initial angular displacement
    omega0 = 0.0     # initial angular velocity
    
    # Bundle initial conditions for ODE solver
    
    y0 = [theta0, omega0]
    # Make time array for solution
    tStop = 200.
    tInc = 0.05
    t = np.arange(0., tStop, tInc)
    
    cnt = 0

    s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    print ('Socket created')

    s.connect((HOST, PORT))

    print('Points numbers: ' + str(len(t)))
    
    from Model import model
    
    print ('Model has been imported')
    
    for x in range(len(t)-1):
        y = odeint(model.function, y0, [t[x], t[x+1]])
        y0 = y[1]
        cnt = cnt + 1
    
        if(cnt>= 10 ):
            s.send(y[0,0])
            cnt = 0;
        for i in range(100000):
            a = i
            
    s.close()
    print ('Socket closed')

In [ ]:
def on_message(client, userdata, message):
    global model_str 
    model_str = str(message.payload.decode("utf-8"))
    print ('Creating model... ' + model_str)
    create_file_model()
    print('Started a solution')
    solver()
    print('Solution complete')

broker_address="10.0.0.101"
client = mqtt.Client("P1") #create new instance
client.on_message=on_message #attach function to callback
client.connect(broker_address) #connect to broker
client.loop_start() #start the loop
client.subscribe("modelo/#")

# Configs for tcp socket
HOST = '10.0.0.101' # Symbolic name, meaning all available interfaces
PORT = 8888 #Arbitrary non-privileged port
model_str = ""

print("Running")

while True:
    time.sleep(1)

Running
Creating model... omega$-omega/Q + np.sin(theta) + d*np.cos(Omega*t)$Q = 10, d = 5, Omega = 1$tau = 0, omega = 0, t = 0
Started a solution
Socket created
Points numbers: 4000
Model has been imported
Socket closed
Solution complete
